In [4]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains, Keys
from automatic_writer import get_property_id
from image_upload import get_map_images, get_ground_plan_images, select_random_images, upload_images_to_blog, get_specific_image
from datetime import datetime, timezone
from bs4 import BeautifulSoup
import tkinter as tk
from tkinter import simpledialog
import time
import json
import os
import logging
import pyperclip

In [34]:
if __name__ == "__main__":
    # 팝업으로 매물번호 입력 받기
    property_id = get_property_id()
    if property_id:
        title, information, description = main(property_id)
    else:
        print("매물번호가 입력되지 않았습니다.")

In [35]:
# MongoDB 연결
client = MongoClient('mongodb://localhost:27017/')
db = client['property_db']

# 모든 컬렉션 조회
collections = db.list_collection_names()

complex_name = None
property_url = None

# 각 컬렉션에서 검색
for collection_name in collections:
    collection = db[collection_name]
    
    # 매물목록 배열 내에서 매물번호로 검색하고 해당 매물의 정보 가져오기
    document = collection.find_one(
        {
            "매물목록": {
                "$elemMatch": {
                    "매물번호": property_id
                }
            }
        },
        {
            "단지명": 1,
            "매물목록.$": 1  # 매칭된 매물 정보만 가져오기
        }
    )
    
    if document:
        complex_name = document.get('단지명')
        # 매물목록 배열에서 첫 번째(매칭된) 매물의 URL 가져오기
        if document.get('매물목록') and len(document['매물목록']) > 0:
            property_url = document['매물목록'][0].get('URL')
            print(f"단지명을 찾았습니다: {complex_name} (컬렉션: {collection_name})")
            print(f"매물 URL: {property_url}")
            break

if not complex_name:
    print("해당 property_id의 단지명을 찾을 수 없습니다.")
if not property_url:
    print("해당 매물의 URL을 찾을 수 없습니다.")

# 연결 종료
client.close()

단지명을 찾았습니다: 삼익 (컬렉션: apt)
매물 URL: https://new.land.naver.com/complexes/1031?ms=37.5215,127.05852,16&a=JGC:PRE&b=A1&e=RETAIL&articleNo=2462850598


In [5]:
chrome_options = Options()
service = Service(r'C:\Users\pqstv\tmttd\MARS\BHL\services\crawling\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)

In [37]:
driver.get('https://blog.naver.com/juka103?Redirect=Write&')

In [38]:
id = 'juka103'
pwd = 'samik9407!'

# 로그인
id_input = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[1]/form/ul/li/div/div[1]/div/div[1]/input')
id_input.click()
pyperclip.copy(id)
actions = ActionChains(driver)
actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
time.sleep(1)
pwd_input = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[1]/form/ul/li/div/div[1]/div/div[2]/input')
pwd_input.click()
pyperclip.copy(pwd)
actions = ActionChains(driver)
actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
time.sleep(1)
# id_input.send_keys('juka103')
# pwd_input = driver.find_element(By.XPATH, '//*[@id="pw"]')
# pwd_input.send_keys('samik9407!')
button = driver.find_element(By.XPATH, '//*[@id="log.login"]')
button.click()
time.sleep(2)
# driver.switch_to.window(driver.window_handles[-1])

In [39]:
driver.switch_to.frame('mainFrame')

In [40]:
# 제목 작성

bb = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "제목")]')))
bb.click()
actions = ActionChains(driver)
actions.send_keys(f"{title}").perform()

In [41]:
# 본문 작성
try:
    # 먼저 본문 영역 컨테이너를 찾습니다
    content_container = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.se-component.se-text.se-l-default'))
    )
    content_container.click()
    time.sleep(1)  # 클릭 후 잠시 대기
    
    # 그 다음 실제 입력 가능한 paragraph를 찾습니다
    content_area = content_container.find_element(By.CSS_SELECTOR, 'p.se-text-paragraph')
    content_area.click()
    actions = ActionChains(driver)
    actions.send_keys(f'{information}').perform()
    time.sleep(2)
except Exception as e:
    print("본문 영역을 찾을 수 없습니다:", str(e))


In [42]:
# 랜덤 이미지 업로드
random_images = select_random_images(complex_name, num_images=3)
success = upload_images_to_blog(driver, random_images)

# 배치도 이미지 업로드
plan_images = get_ground_plan_images(complex_name)
success = upload_images_to_blog(driver, plan_images)

C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\HSCP_3.jpg 업로드 시도 중...
업로드 완료: C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\HSCP_3.jpg
C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\HSCPFCTS_4.jpg 업로드 시도 중...
업로드 완료: C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\HSCPFCTS_4.jpg
C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\HSCP_11.jpg 업로드 시도 중...
업로드 완료: C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\HSCP_11.jpg
모든 사진 업로드가 완료되었습니다.
C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\GRDPLAN_1.jpg 업로드 시도 중...
업로드 완료: C:\Users\pqstv\Desktop\real_estate_automization\images\삼익\GRDPLAN_1.jpg
모든 사진 업로드가 완료되었습니다.


In [43]:
actions = ActionChains(driver)
actions.send_keys(f'{description}').perform()
time.sleep(2)

In [45]:
actions.send_keys(Keys.ENTER).perform()
actions.send_keys(Keys.ENTER).perform()
final_text = '-더 자세한 정보가 궁금하다면? 아래 링크 클릭!'
actions.send_keys(f'{final_text}').perform()
actions.send_keys(Keys.ENTER).perform()

In [7]:
print(driver.capabilities)

{'acceptInsecureCerts': False, 'browserName': 'chrome', 'browserVersion': '131.0.6778.205', 'chrome': {'chromedriverVersion': '131.0.6778.108 (3b014839fbc4fb688b2f5af512d6ce312ad208b1-refs/branch-heads/6778@{#2393})', 'userDataDir': 'C:\\Users\\pqstv\\AppData\\Local\\Temp\\scoped_dir9044_1578043341'}, 'fedcm:accounts': True, 'goog:chromeOptions': {'debuggerAddress': 'localhost:52867'}, 'networkConnectionEnabled': False, 'pageLoadStrategy': 'normal', 'platformName': 'windows', 'proxy': {}, 'setWindowRect': True, 'strictFileInteractability': False, 'timeouts': {'implicit': 0, 'pageLoad': 300000, 'script': 30000}, 'unhandledPromptBehavior': 'dismiss and notify', 'webauthn:extension:credBlob': True, 'webauthn:extension:largeBlob': True, 'webauthn:extension:minPinLength': True, 'webauthn:extension:prf': True, 'webauthn:virtualAuthenticators': True}


In [46]:
 # 링크 버튼 찾기 및 클릭
link_button = driver.find_element(By.CSS_SELECTOR, 'button[data-name="oglink"]')
driver.execute_script("arguments[0].click();", link_button)
time.sleep(1)

actions.send_keys(f'{property_url}').perform()
time.sleep(1)
actions.send_keys(Keys.ENTER).perform()
# 확인 버튼이 클릭 가능할 때까지 대기 후 JavaScript로 클릭
confirm_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.se-popup-button-confirm'))
)
# 버튼으로 스크롤
driver.execute_script("arguments[0].scrollIntoView(true);", confirm_button)
time.sleep(1)
# 클릭 실행
driver.execute_script("arguments[0].click();", confirm_button)

In [47]:
# 부동산 이미지 업로드
image_path = get_specific_image(r"C:\Users\pqstv\Desktop\real_estate_automization\images\shop.jpg")
success = upload_images_to_blog(driver, [image_path])

# 명함 이미지 업로드
image_path = get_specific_image(r"C:\Users\pqstv\Desktop\real_estate_automization\images\card.jpg")
success = upload_images_to_blog(driver, [image_path])

C:\Users\pqstv\Desktop\real_estate_automization\images\shop.jpg 업로드 시도 중...
업로드 완료: C:\Users\pqstv\Desktop\real_estate_automization\images\shop.jpg
모든 사진 업로드가 완료되었습니다.
C:\Users\pqstv\Desktop\real_estate_automization\images\card.jpg 업로드 시도 중...
업로드 완료: C:\Users\pqstv\Desktop\real_estate_automization\images\card.jpg
모든 사진 업로드가 완료되었습니다.


In [48]:
# 발행
try:
    # 발행 버튼 찾기
    publish_button = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.publish_btn__m9KHH'))
    )
    # JavaScript로 클릭 실행
    driver.execute_script("arguments[0].click();", publish_button)
    time.sleep(1)

    # 카테고리 선택
    category_button = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/div/div[3]/div[2]/div/div/div/div[1]/div/div/button'))
    )
    driver.execute_script("arguments[0].click();", category_button)
    time.sleep(1)

    # 특정 카테고리 선택 (예: "매물정보")
    # category_item = WebDriverWait(driver, 3).until(
    #     EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="categoryBtn_7"]'))
    # )
    
        # 또는 label을 통해 선택
    category_item = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for="7_매물정보"]'))
    )
    driver.execute_script("arguments[0].click();", category_item)
    time.sleep(1)

    # 최종 발행 버튼
    final_publish = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/div/div[3]/div[2]/div/div/div/div[8]/div/button'))
    )
    driver.execute_script("arguments[0].click();", final_publish)
    time.sleep(2)

except Exception as e:
    print("발행 과정에서 오류 발생:", str(e))